In [1]:
# On terminal: conda activate python38

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ipynb.fs.defs.functions as fct
import pickle

import warnings
warnings.filterwarnings("error")

In [9]:
# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

# Fit mod1 for each participant

In [14]:
# Model settings

# Functions
value_fct = fct.rescorla_wagner
dec_fct = fct.my_softmax

# Store everything
mod_info = {}
mod_info['name'] = 'model1'
mod_info['value_fct'] = value_fct.__name__
mod_info['dec_fct'] = dec_fct.__name__
mod_info['param_names'] = ['v0', 'alpha', 'beta']
#print(mod['value_fct'].__name__)

# save
all_users_folder = 'data/all_users/mod1/'
file_name = all_users_folder+'mod_parameters.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(mod_info, f)

In [15]:
ID = '001'

user_folder = 'data/user_' + ID + '/'
df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
isHit_all_cues, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

Exception ignored in: <function Socket.__del__ at 0x7fae3075f550>
Traceback (most recent call last):
  File "/Users/magdadubois/opt/anaconda3/envs/python38/lib/python3.8/site-packages/zmq/sugar/socket.py", line 97, in __del__
    warnings.warn(


In [12]:
run_ = True

# Folder
all_users_folder = 'data/all_users/mod1/'
    
if run_:
    
    # Parameter range for initial guess 
    # ['v0', 'alpha', 'beta']
    param_lower_bound = [-10, 0, 0]
    param_upper_bound = [ 10, 1, 3]

    # Fit
    all_users = {}
    p_hit_per_trial = pd.DataFrame([])
    ev_per_trial = pd.DataFrame([])
    for n_part,ID in enumerate(uniqueIDs): 

        # Get data
        user_folder = 'data/user_' + ID + '/'
        df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
        isHit_all_cues, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

        # Create a new Model object
        mod = fct.Model(mod_name = mod_info['name'],
                     value_fct = value_fct, 
                     dec_fct = dec_fct, 
                     param_names = mod_info['param_names'])

        # Input data to model
        mod.set_data(ID, fbs_all_cues, isHit_all_cues, trialNo_all_cues)

        # Fit model
        mod.fit(param_lower_bound, param_upper_bound, n_iterations=5)

        # Nested dictionnary user data
        all_users[n_part] = {}
        all_users[n_part]['ID']=mod.ID
        all_users[n_part]['nLL']=mod.nLL
        all_users[n_part]['Ntrials']=mod.Ntrials
        all_users[n_part]['Nparams']=len(mod.param_names)
        for i in range(0,len(mod.param_names)):
            all_users[n_part][mod.param_names[i]]=mod.param_values[i]
        
        # Concatenated model predictions: p hit
        tmp = pd.DataFrame(mod.p_hit).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        p_hit_per_trial = pd.concat([tmp, p_hit_per_trial], axis=0)
        
        # Concatenated model predictions: EVs
        tmp = pd.DataFrame(mod.v).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        ev_per_trial = pd.concat([tmp, ev_per_trial], axis=0)
    
    # Save mod LLs and parameter values
    mod_fit = pd.DataFrame(all_users).transpose()
    mod_fit.to_pickle(all_users_folder+'mod_param_fits.pkl')
    
    # Save mod predictions
    p_hit_per_trial = p_hit_per_trial.sort_values(by='ID').reset_index(drop=True)
    p_hit_per_trial.to_pickle(all_users_folder+'mod_p_hit_per_trial.pkl')
    ev_per_trial = ev_per_trial.sort_values(by='ID').reset_index(drop=True)
    ev_per_trial.to_pickle(all_users_folder+'mod_ev_per_trial.pkl')
    

In [13]:
mod_fit

,ID,nLL,Ntrials,Nparams,v0,alpha,beta
0,001,54.352932,112,3,-0.470614,0.015483,3.0
1,003,53.289788,112,3,0.141746,0.026828,3.0
2,006,54.597407,112,3,0.564168,0.195576,0.979622
3,007,50.411542,112,3,0.626299,0.041902,1.985282
4,008,44.242888,112,3,0.343681,0.033001,3.0
...,...,...,...,...,...,...,...
165,453,51.765799,112,3,8.059598,0.294032,0.789652
166,462,45.068864,112,3,0.080988,0.036538,3.0
167,469,10.775145,112,3,0.124262,0.128403,3.0
168,470,50.640691,112,3,0.162665,0.027244,3.0


In [13]:
# Input softmax
vt_all_cues = mod.v

In [14]:
# output softmax
mod.p_hit

{'HR': array([0.1959437 , 0.23907957, 0.25098667, 0.21270819, 0.20835578,
        0.20413521, 0.24836901, 0.26042999, 0.2210042 , 0.26735808,
        0.31860058, 0.37374349, 0.38657121, 0.44464059, 0.50333571,
        0.56104051, 0.61629131, 0.62549765, 0.56776981, 0.57810146,
        0.63236056, 0.68270027, 0.66993603, 0.67773564, 0.62237132,
        0.67351784, 0.72014413, 0.70766952]),
 'LR': array([0.1959437 , 0.23907957, 0.25098667, 0.26308831, 0.3139361 ,
        0.26856286, 0.22817188, 0.27537026, 0.32732386, 0.34013085,
        0.33205613, 0.34487841, 0.35772673, 0.41485432, 0.40490752,
        0.41759689, 0.40758795, 0.35425097, 0.30505119, 0.29795393,
        0.31059478, 0.36522775, 0.42264194, 0.48129425, 0.49318817,
        0.55117578, 0.60694565, 0.61638064]),
 'HP': array([0.1959437 , 0.16489736, 0.20345084, 0.17136537, 0.18166164,
        0.15263589, 0.12788552, 0.16005606, 0.13419193, 0.14318613,
        0.11987398, 0.10019988, 0.09908171, 0.09799172, 0.09692907,
      

In [33]:
import scipy

In [34]:
beta = 2

p_hit_all_cues = dict.fromkeys(vt_all_cues.keys())

for cue, vts in vt_all_cues.items():
    p_hit = []
    for vt in vts:
        Q_hit = vt * beta
        Q_miss = 0
        probs = scipy.special.softmax([Q_hit, Q_miss])
        p_hit.append(probs[0])
    
    p_hit_all_cues[cue] = np.array(p_hit)
    

In [35]:
p_hit_all_cues

{'HR': array([0.28065237, 0.31608519, 0.32543514, 0.29474439, 0.2911284 ,
        0.28759408, 0.32339351, 0.33273891, 0.3015595 , 0.33803898,
        0.37594235, 0.41480735, 0.42364399, 0.46300951, 0.50222383,
        0.5408067 , 0.57832247, 0.58466809, 0.54533478, 0.55230536,
        0.58942076, 0.62499542, 0.61584058, 0.62142242, 0.58250949,
        0.61839927, 0.65251616, 0.64322628]),
 'LR': array([0.28065237, 0.31608519, 0.32543514, 0.33477822, 0.37257459,
        0.33895583, 0.30737041, 0.34411051, 0.38220271, 0.39131038,
        0.3855792 , 0.39466311, 0.40367807, 0.44292758, 0.43617376,
        0.44478506, 0.43799651, 0.40124742, 0.36611833, 0.36091997,
        0.37015309, 0.40890431, 0.44819701, 0.48752626, 0.49545863,
        0.53418366, 0.57191311, 0.57838389]),
 'HP': array([0.28065237, 0.2532246 , 0.28701833, 0.25909949, 0.26826953,
        0.2418219 , 0.2175746 , 0.24876546, 0.22392672, 0.23277338,
        0.20931114, 0.18795745, 0.18669249, 0.18545353, 0.18423998,
      

In [14]:
mod.v['HR'][0]

-0.4706139660184445

In [16]:
x = [mod.v['HR'][0], 0]
x

[-0.4706139660184445, 0]

In [20]:
from scipy.special import softmax

In [21]:
softmax(x)

array([0.38447094, 0.61552906])

In [26]:
beta = 4
Q_hit = mod.v['HR'][0] * beta
Q_miss = 0
softmax([Q_hit, Q_miss])


array([0.13210704, 0.86789296])

In [ ]:
    # Free parameters
    beta = param_values[param_names.index('beta')]
    
    # Initialise empty dicitonnary
    p_hit_all_cues = dict.fromkeys(vt_all_cues.keys())
    
    # Iterate over cuesss
    for cue, vt in vt_all_cues.items():
                
        x = beta*vt
        
        try:
            p_hit =  np.exp(x)/(np.exp(x)+1)
            
        except RuntimeWarning:
            # to avoid overflow errors
            expon_bound = 700
            p_hit = [1 if el>expon_bound else (np.exp(el)/(np.exp(el)+1)) for el in x]
            
        p_hit_all_cues[cue] = p_hit
    
    return p_hit_all_cues